In [1]:
import cv2
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog
from sklearn.model_selection import GridSearchCV

In [3]:
from simple_image_download import simple_image_download as simp 

response = simp.simple_image_download
keywords = ["mercedes logo","volkswagen logo","toyota logo"]

for kw in keywords:
    response().download(kw , 200)

[========================================================================] 100%
[========================================================================] 100%
[========================================================================] 100%


In [10]:
# 1. Charger les images
image_dir_1 = "./simple_images/mercedes_logo"
image_dir_2 = "./simple_images/volkswagen_logo"
image_dir_3 = "./simple_images/toyota_logo"


images = []
labels = []

def read_data(images_dir , category ):
    global images,labels
    for filename in os.listdir(images_dir):
        img = cv2.imread(os.path.join(images_dir, filename))
        label = category  # label est le nom de la marque de la voiture
        images.append(img)
        labels.append(label)

read_data(image_dir_1 , "mercedes" )
read_data(image_dir_2 , "volkswagen" )
read_data(image_dir_3 , "toyota" )
        


In [11]:

# 2. Prétraitement des images
gray_images = []
for img in images:
    resized_img = cv2.resize(img, (200,200))
    gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
    gray_images.append(gray)


In [12]:
# 3. Extraction de fonctionnalités HOG
features = []
for img in gray_images:
    fd, hog_image = hog(img, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, channel_axis=None)
    features.append(fd)

In [13]:
print(len(features))
print(features[0].shape)

516
(20736,)


In [14]:
# 4. Construction des vecteurs de caractéristiques
X = np.vstack(features)
y = np.array(labels)

In [15]:
X.shape

(516, 20736)

In [16]:
# Entrainement du modèle ( classification des logos )
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
svm_model = SVC()

parameters = [{'gamma':[0.01, 0.001,0.002],'C':[0.1,1,10,100]}]

grid_search = GridSearchCV(svm_model,parameters)

grid_search.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [0.1, 1, 10, 100],
                          'gamma': [0.01, 0.001, 0.002]}])

In [19]:
# Affichage des meilleurs hyperparamètres et du score correspondant
print(f"Meilleurs hyperparamètres : {grid_search.best_params_}")
print(f"Score de validation croisée : {grid_search.best_score_}")

Meilleurs hyperparamètres : {'C': 100, 'gamma': 0.001}
Score de validation croisée : 0.9199235968263297


In [20]:
best_estimator = grid_search.best_estimator_

In [21]:
y_pred = best_estimator.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

    mercedes       0.90      1.00      0.95        38
      toyota       0.96      0.93      0.95        28
  volkswagen       0.94      0.87      0.90        38

    accuracy                           0.93       104
   macro avg       0.94      0.93      0.93       104
weighted avg       0.93      0.93      0.93       104

Accuracy: 0.9326923076923077


In [23]:
import joblib

joblib.dump(best_estimator,"model_reconnaitre_logo.joblib")

['model_reconnaitre_logo.joblib']

In [24]:
dir_test_1 = "C:/Users/Hp/Desktop/fst ML/projet/archive/Car_Brand_Logos/Test/mercedes/mrc93.jpg"
dir_test_1 = "C:/Users/Hp/Desktop/fst ML/projet/archive/Car_Brand_Logos/Test/hyundai/hnd10.jpg"
dir_test_1 = "C:/Users/Hp/Desktop/fst ML/projet/archive/Car_Brand_Logos/Test/mazda/mazda10.jpg"
dir_test_1 = "C:/Users/Hp/Desktop/fst ML/projet/archive/Car_Brand_Logos/Test/toyota_/toyota10.jpg"

img_test_1 = cv2.imread(dir_test_1)

resized_img_test_1 = cv2.resize(img_test_1, (200, 200))
gray_test_1 = cv2.cvtColor(resized_img_test_1, cv2.COLOR_BGR2GRAY)

fd_test_1, hog_image = hog(gray_test_1, orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, multichannel=False)

fd_test_1 = fd_test_1.reshape(1,-1)
print(fd_test_1.shape)

import matplotlib.pyplot as plt
from skimage import exposure

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), sharex=True, sharey=True)
ax1.imshow(resized_img_test_1, cmap=plt.cm.gray)
ax1.set_title('Image originale')
# Réajuster le contraste pour l'affichage de l'image HOG
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Descripteurs HOG')
plt.show()

y_pred_1 = best_estimator.predict(fd_test_1)
print(y_pred_1)


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
